# Bayes ingenuo tarea

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo. Cómo tarea quedó agregar el entrenamiento con una tercera clase. Este documento es el entregable del equipo 7: Alan Almora y Samuel Leidenberger

In [82]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [83]:
usePackage('R.utils')
usePackage('tm')
usePackage('ramify')

## Descarga los datos 

In [84]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [85]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

[1] TRUE

In [86]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [87]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [88]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt")
  lineas <- readLines(fd, warn=FALSE)
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n"))
}

In [89]:
# Creamos variables con los directorios donde se encuentran los datos
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### Spam

In [90]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
archivos_correos_spam_training <- archivos_correos_spam[1:201]
archivos_correos_spam_testing <- archivos_correos_spam[201:250]

todo_spam <- sapply(archivos_correos_spam_training,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [91]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:201]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[201:250]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

### Hard ham

In [92]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")] #[1:250]

archivos_correos_hard_ham <- archivos_correos_hard_ham[sample(1:length(archivos_correos_hard_ham))]
archivos_correos_hard_ham_training <- archivos_correos_hard_ham[1:201]
archivos_correos_hard_ham_testing <- archivos_correos_hard_ham[201:250]

todo_hard_ham <- sapply(archivos_correos_hard_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_hardham, p)))

todo_hard_ham <- enc2utf8(todo_hard_ham)

## Preparación de corpus y bolsa de palabras

In [93]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 2)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [94]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [95]:
head(df_spam)

terminos,frecuencia,densidad,ocurrencias
auto,5,9.687106e-05,0.01990050
can,265,5.134166e-03,0.40298507
cards,18,3.487358e-04,0.05970149
compete,6,1.162453e-04,0.02985075
consolidation,9,1.743679e-04,0.02985075
consultation,6,1.162453e-04,0.02487562


### Easy ham

In [96]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)
head(df_easy_ham)

terminos,frecuencia,densidad,ocurrencias
alexandre,2,6.451821e-05,0.009950249
angles,4,1.290364e-04,0.019900498
anglesaminvestmentscom,1,3.225911e-05,0.004975124
arafat,4,1.290364e-04,0.019900498
audiofile,1,3.225911e-05,0.004975124
awful,2,6.451821e-05,0.009950249


### Hard ham


In [97]:
hard_ham_TDM <- obtiene_TermDocumentMatrix(todo_hard_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento hard ham
matriz_hard_ham <- as.matrix(hard_ham_TDM)

conteos_hard_ham <- rowSums(matriz_hard_ham)
df_hard_ham <- data.frame(cbind(names(conteos_hard_ham),
                            as.numeric(conteos_hard_ham)),
                      stringsAsFactors = FALSE)
names(df_hard_ham) <- c("terminos", "frecuencia")
df_hard_ham$frecuencia <- as.numeric(df_hard_ham$frecuencia)
ocurrencias_hard_ham <- sapply(1:nrow(matriz_hard_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_hard_ham[i, ] > 0)) / ncol(matriz_hard_ham)
                           })
densidad_hard_ham <- df_hard_ham$frecuencia/sum(df_hard_ham$frecuencia,na.rm = TRUE)

df_hard_ham <- transform(df_hard_ham,
                     densidad = densidad_hard_ham,
                     ocurrencias = ocurrencias_hard_ham)
head(df_hard_ham)

terminos,frecuencia,densidad,ocurrencias
address,257,1.104066e-03,0.686567164
also,232,9.966663e-04,0.517412935
alt,1106,4.751349e-03,0.328358209
aufontfont,8,3.436780e-05,0.004975124
available,148,6.358044e-04,0.318407960
availablebr,13,5.584768e-05,0.064676617


## Cálculo de probabilidad a posteriori

In [98]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c))
  }
  else
  {
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

## Clasificación

In [104]:
bi_clasifica<- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  
  return (ifelse(hard_ham_spam_prueba > hard_ham_ham_prueba,
                        TRUE,
                        FALSE))
}

In [105]:
tri_clasifica<- function(trayectoria, archivos) {

  densidades_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  densidades_easy_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  densidades_hard_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_hard_ham))
                                       
    matriz_densidades <- matrix(c(densidades_spam_prueba, densidades_easy_ham_prueba, densidades_hard_ham_prueba), ncol=3)
    return(argmax(matriz_densidades,rows=TRUE))
    
         
}

In [106]:
bi_hard_ham_res <- bi_clasifica(trayectoria_hardham, archivos_correos_hard_ham_testing)
bi_easy_ham_res <- bi_clasifica(trayectoria_easyham, archivos_correos_easy_ham_testing)
bi_spam_res     <- bi_clasifica(trayectoria_spam,    archivos_correos_spam_testing)

tri_hard_ham_res <- tri_clasifica(trayectoria_hardham, archivos_correos_hard_ham_testing)
tri_easy_ham_res <- tri_clasifica(trayectoria_easyham, archivos_correos_easy_ham_testing)
tri_spam_res     <- tri_clasifica(trayectoria_spam,    archivos_correos_spam_testing)



## Resultados

### Dos clases (easy - spam)

In [110]:
table(bi_easy_ham_res)
table(bi_spam_res)
table(bi_hard_ham_res)

bi_easy_ham_res
FALSE 
   50 

bi_spam_res
FALSE  TRUE 
    5    45 

bi_hard_ham_res
FALSE  TRUE 
   25    25 

### Tres clases (spam - easy - hard)

In [111]:
table(tri_easy_ham_res)
table(tri_spam_res)
table(tri_hard_ham_res)

tri_easy_ham_res
 2  3 
49  1 

tri_spam_res
 1  2  3 
43  5  2 

tri_hard_ham_res
 1  2  3 
 3 11 36 

## Conclusiones

### cambios en relación a la biclasificación

1. easy ham - el modelo tiene misma efectividad en clasificar easy ham, curiosamente, a veces lo clasifica como hard ham esto se justifica con intersecciones atípicamente altas entre palabras de los mails.
2. spam - el modelo sufre un poco en la clasificación de spam, igualmente dadas las intersecciones que cuenta con hard_ham.
3. hard ham - claramente hay una mejoría en la clasificación de hard_ham, esto dado que logra compensar las intersecciones con spam gracias al entrenamiento con su mismo conjunto 
